In [ ]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import scipy.io
from scipy.signal import welch, freqz, butter, filtfilt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importing Data

Note: Importing only data sets 1 and 3 because 2,4,5,6 are poor data quality

In [ ]:
#Import data - event positions
evpos1_1 = pd.read_csv('/content/drive/My Drive/Data/s1_s1/sub1_s1_evpos.csv', header=None, engine='python')
evpos1_2 = pd.read_csv('/content/drive/My Drive/Data/s1_s2/sub1_s2_evpos.csv', header=None, engine='python')
evpos3_1 = pd.read_csv('/content/drive/My Drive/Data/s3_s1/sub3_s1_evpos.csv', header=None, engine='python')
evpos3_2 = pd.read_csv('/content/drive/My Drive/Data/s3_s2/sub3_s2_evpos.csv', header=None, engine='python')
evpos1_1 = np.squeeze(evpos1_1.values.tolist())
evpos1_2 = np.squeeze(evpos1_2.values.tolist())
evpos3_1 = np.squeeze(evpos3_1.values.tolist())
evpos3_2 = np.squeeze(evpos3_2.values.tolist())

In [2]:
# Make all arrays the same size
biggest = max(  evpos1_1.shape[1],
                evpos1_2.shape[1],
                evpos3_1.shape[1],
                evpos3_2.shape[1],)

evpos1_1 = np.append(evpos1_1, np.zeros((10, biggest - evpos1_1.shape[1])), axis=1)
evpos1_2 = np.append(evpos1_2, np.zeros((10, biggest - evpos1_2.shape[1])), axis=1)
evpos3_1 = np.append(evpos3_1, np.zeros((10, biggest - evpos3_1.shape[1])), axis=1)
evpos3_2 = np.append(evpos3_2, np.zeros((10, biggest - evpos3_2.shape[1])), axis=1)

NameError: name 'evpos1_1' is not defined

In [1]:
evpos = np.concatenate((evpos1_1,
                        evpos1_2,
                        evpos3_1,
                        evpos3_2), axis=0)

NameError: name 'np' is not defined

In [ ]:
# Import data - event type
evtype1_1 = pd.read_csv('/content/drive/My Drive/Data/s1_s1/sub1_s1_evtype.csv', header=None, engine='python')
evtype1_2 = pd.read_csv('/content/drive/My Drive/Data/s1_s2/sub1_s2_evtype.csv', header=None, engine='python')
evtype3_1 = pd.read_csv('/content/drive/My Drive/Data/s3_s1/sub3_s1_evtype.csv', header=None, engine='python')
evtype3_2 = pd.read_csv('/content/drive/My Drive/Data/s3_s2/sub3_s2_evtype.csv', header=None, engine='python')
evtype1_1 = np.squeeze(evtype1_1.values.tolist())
evtype1_2 = np.squeeze(evtype1_2.values.tolist())
evtype3_1 = np.squeeze(evtype3_1.values.tolist())
evtype3_2 = np.squeeze(evtype3_2.values.tolist())

In [ ]:
# Make all data arrays the same size
biggest = max(  evtype1_1.shape[1],
                evtype1_2.shape[1],
                evtype3_1.shape[1],
                evtype3_2.shape[1])
evtype1_1 = np.append(evtype1_1, np.zeros((10, biggest - evtype1_1.shape[1])), axis=1)
evtype1_2 = np.append(evtype1_2, np.zeros((10, biggest - evtype1_2.shape[1])), axis=1)
evtype3_1 = np.append(evtype3_1, np.zeros((10, biggest - evtype3_1.shape[1])), axis=1)
evtype3_2 = np.append(evtype3_2, np.zeros((10, biggest - evtype3_2.shape[1])), axis=1)

In [ ]:
evtype = np.concatenate((   evtype1_1,
                            evtype1_2,
                            evtype3_1,
                            evtype3_2), axis=0)

In [ ]:
# Import EEG data
data1_1 = pd.read_csv('/content/drive/My Drive/Data/s1_s1/sub1_s1_FCz_data.csv', header=None, engine='python')
data1_2 = pd.read_csv('/content/drive/My Drive/Data/s1_s2/sub1_s2_FCz_data.csv', header=None, engine='python')
data3_1 = pd.read_csv('/content/drive/My Drive/Data/s3_s1/sub3_s1_FCz_data.csv', header=None, engine='python')
data3_2 = pd.read_csv('/content/drive/My Drive/Data/s3_s2/sub3_s2_FCz_data.csv', header=None, engine='python')
data1_1 = np.squeeze(data1_1.values.tolist())
data1_2 = np.squeeze(data1_2.values.tolist())
data3_1 = np.squeeze(data3_1.values.tolist())
data3_2 = np.squeeze(data3_2.values.tolist())

In [ ]:
# Make all data arrays the same size
biggest = max(  data1_1.shape[1],
                data1_2.shape[1],
                data3_1.shape[1],
                data3_2.shape[1])
data1_1 = np.append(data1_1, np.zeros((10, biggest - data1_1.shape[1])), axis=1)
data1_2 = np.append(data1_2, np.zeros((10, biggest - data1_2.shape[1])), axis=1)
data3_1 = np.append(data3_1, np.zeros((10, biggest - data3_1.shape[1])), axis=1)
data3_2 = np.append(data3_2, np.zeros((10, biggest - data3_2.shape[1])), axis=1)

In [ ]:
data = np.concatenate(( data1_1,
                        data1_2,
                        data3_1,
                        data3_2), axis=0)

In [ ]:
# Normalise data
norm_data = np.zeros(data.shape)
end = np.zeros(data.shape[0]).astype(int)
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        if data[i,j] == 0:
            end[i] = j
            break
        else:
            end[i] = data.shape[1]
    normalise = np.mean(data[i,0:end[i]])
    norm_data[i,0:end[i]] = data[i,0:end[i]] - normalise

# Filtering

In [ ]:
fs = 512

def butter_bandpass(lowcut, highcut, fs, order=2):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=2):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y


def plot_spectrum(data, sf, window_sec, band=None, dB=False):
    """Plot the periodogram, Welch's and multitaper PSD.

    Parameters:
    data : 1d-array
        Input signal in the time-domain.
    sf : float
        Sampling frequency of the data.
    band : list
        Lower and upper frequencies of the band of interest.
    window_sec : float
        Length of each window in seconds for Welch's PSD
    dB : boolean
        If True, convert the power to dB.
    """
    sns.set(style="white", font_scale=1.2)
    # compute PSD
    freqs_welch, psd_welch = welch(data, sf, nperseg=window_sec*sf)
    sharey = False

    # convert power to decibels (dB = 10 * log10(power))
    if dB:
        psd_welch = 10 * np.log10(psd_welch)
        sharey = True
    
    return freqs_welch, psd_welch

Find PSD

In [ ]:
# PSD range
low = 1
high = 10

# Create arrays for freqs and PSDs
test1, test2 = plot_spectrum(data[0,:], fs, (32), [low, high], dB=True)
all_freqs = np.zeros([test1.size, data.shape[0]])
all_psd = np.zeros([test2.size, data.shape[0]])

for i in range(data.shape[0]):
    all_freqs[:,i], all_psd[:,i] = plot_spectrum(data[i,:], fs, (32), [low, high], dB=True)

# Find means
mean_freqs = np.mean(all_freqs, axis=1)
mean_psd = np.mean(all_psd, axis=1)

# Identify area of interest
idx_mu = np.logical_and(mean_freqs >= low, mean_freqs <= high)

# Plot
plt.figure(figsize=(7, 4))
plt.plot(mean_freqs, mean_psd, lw=1, color='k')
plt.fill_between(mean_freqs, mean_psd, where=idx_mu, color='skyblue')
plt.xlabel('Frequency (Hz)', fontsize=20)
plt.ylabel('Decibels (dB / Hz)', fontsize=20)
plt.xlim([1, 60])
plt.ylim([0, mean_psd.max()/2])
plt.title("PSD of channel", fontsize=30)

In [ ]:
# Pass data through butterworth and bandpass filter

data_filt = np.zeros((data.shape[0], data.shape[1]))
for i in range(data.shape[0]):
    data_filt[i,:] = (butter_bandpass_filter(data[i,:].T, low, high, fs).T)

Separate data into positive/negative trials 

In [3]:
correct_trials_filt_all = np.zeros([2,fs])
error_trials_filt_all = np.zeros([2,fs])

# Identify correct / error trials from `evtype` and slot them into respective arrays
for i in range(evtype.shape[0]):
    
    for j in range(evtype.shape[1]):
        if evtype[i,j] == 5 or evtype[i,j] == 10:
            correct_trials_filt_all = np.append(correct_trials_filt_all, [data_filt[i, int(evpos[i,j]):int(evpos[i,j] + fs)]], axis=0)
        elif evtype[i,j] == 6 or evtype[i,j] == 9:
            error_trials_filt_all = np.append(error_trials_filt_all, [data_filt[i, int(evpos[i,j]):int(evpos[i,j] + fs)]], axis=0)

correct_trials_filt_all = correct_trials_filt_all[2:,:]
error_trials_filt_all = error_trials_filt_all[2:,:]

# Normalise
for i in range(correct_trials_filt_all.shape[0]):
  ave = np.average(correct_trials_filt_all[i,:])
  correct_trials_filt_all[i,:] = correct_trials_filt_all[i,:] - ave

for i in range(error_trials_filt_all.shape[0]):
  ave = np.average(error_trials_filt_all[i,:])
  error_trials_filt_all[i,:] = error_trials_filt_all[i,:] - ave

mean_correct_filt_all = np.mean(correct_trials_filt_all, axis=0)
mean_error_filt_all = np.mean(error_trials_filt_all, axis=0)

NameError: name 'np' is not defined

# Plot

In [ ]:
start = 0
seconds = 1
stop = start + (fs*seconds) 
t = [((i/fs)*1000) for i in range(start, stop)]

plt.figure(figsize=(20,8))
plt.title('Mean of ERRP in correct vs. erroneous trials', pad = 30,
                                                          fontsize=30,
                                                          bbox=dict(facecolor='black', alpha=0.05, edgecolor='black'),
                                                          fontfamily='serif',
                                                          color='black',
                                                          ha='center')
plt.plot(t, mean_correct_filt_all, label = "Absence of ERRP", 
                                   lw=2.5,
                                   c='#00A36C')
plt.plot(t, mean_error_filt_all, label = 'Error-related potential', 
                                 lw=2.5,
                                 c='#FF7F50')

plt.axvline(x = 158, ls='--', lw=1.25)
plt.axvline(x = 572, ls='--', lw=1.25)

plt.legend(borderaxespad = 8)

plt.xlim([0, 800])
plt.ylim([-10, 10])
plt.xlabel("Time (ms)", labelpad=15,
                        fontsize=20,
                        fontfamily='monospace',
                        color='black',
                        ha='center')
plt.ylabel("uV", labelpad=15,
                 fontsize=18,
                 fontfamily='monospace',
                 color='black',
                 rotation=0,
                 va='center')

# Feature extraction

In [ ]:
from scipy import stats

def mean(data):
    return np.mean(data)
    
def std(data):
    return np.std(data)

def ptp(data):
    return np.ptp(data)

def var(data):
        return np.var(data)

def minim(data):
      return np.min(data)

def maxim(data):
      return np.max(data)

def argminim(data):
      return np.argmin(data)

def argmaxim(data):
      return np.argmax(data)

def mean_square(data):
      return np.mean(data**2)

def rms(data): #root mean square
      return  np.sqrt(np.mean(data**2))  

def abs_diffs_signal(data):
    return np.sum(np.abs(np.diff(data)))

def skewness(data):
    return stats.skew(data)

def kurtosis(data):
    return stats.kurtosis(data)

In [ ]:
def concatenate_features(data):
    return np.array(((  mean(data),
                        std(data),
                        ptp(data),
                        var(data),
                        minim(data),
                        maxim(data),
                        argminim(data),
                        argmaxim(data),
                        mean_square(data),
                        rms(data),
                        abs_diffs_signal(data),
                        skewness(data),
                        kurtosis(data))))

In [ ]:
features_correct = np.zeros([correct_trials_filt_all.shape[0],13])
for i in range(correct_trials_filt_all.shape[0]):
    features_correct[i,:] = concatenate_features(correct_trials_filt_all[i,:])

features_error = np.zeros([error_trials_filt_all.shape[0],13])
for i in range(error_trials_filt_all.shape[0]):
    features_error[i,:] = concatenate_features(error_trials_filt_all[i,:])

# Machine learning

In [4]:
# Split data into features `x` and labels `y`

# Create arrays for filtered data (0 = correct, 1 = error) 
a = np.zeros((correct_trials_filt_all.shape[0],correct_trials_filt_all.shape[1]+1))
b = np.ones((error_trials_filt_all.shape[0],error_trials_filt_all.shape[1]+1))

# Fill arrays except for last column (to record labels)
a[:,0:-1] = correct_trials_filt_all
b[:,0:-1] = error_trials_filt_all

# Shuffle data for training sets
x_and_y = np.concatenate((a[:b.shape[0]], b), axis=0)
np.random.shuffle(X_and_y)

# create feature (`x`) and label (`y`) arrays
y = x_and_y[:,-1]
x = x_and_y[:,0:-1]

# create training sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,
                                                    y, 
                                                    test_size=0.2,
                                                    shuffle=False) # Don't mix up our shit

NameError: name 'np' is not defined

In [5]:
# Create model and test training data

from sklearn.svm import SVC
model = SVC(kernel='rbf')
model.fit(X_train, y_train)
print("Accuracy: ", model.score(X_test, y_test))

NameError: name 'X_train' is not defined